In [3]:
import numpy as np
import pandas as pd

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
```

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).


### Preparing the dataset 

For this homework, we only want to use a subset of data. This is the same subset we used in homework #2.

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Preparation:

* Fill missing values with zeros.
* Apply the log tranform to `median_house_value`.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframe into matrices.

In [4]:
data = pd.read_csv("data/housing.csv")
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
data.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [16]:
#First, keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'
df = data.loc[(data.ocean_proximity == '<1H OCEAN') | (data.ocean_proximity == 'INLAND')]

In [17]:
df.ocean_proximity.value_counts()

ocean_proximity
<1H OCEAN    9136
INLAND       6551
Name: count, dtype: int64

In [19]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [20]:
# Fill missing values with zeros.
df = df.fillna(0)

In [21]:
df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [22]:
# before
df.median_house_value.head()

701    431000.0
830    217000.0
859    247600.0
860    283500.0
861    216900.0
Name: median_house_value, dtype: float64

In [23]:
# Apply the log tranform to median_house_value
df['median_house_value'] = np.log1p(df['median_house_value'])

In [24]:
df.median_house_value.head()

701    12.973866
830    12.287657
859    12.419574
860    12.554971
861    12.287196
Name: median_house_value, dtype: float64

In [25]:
# Do train/validation/test split with 60%/20%/20% distribution.
from sklearn.model_selection import train_test_split

In [28]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)


In [29]:
len(df_train_full), len(df_train), len(df_val), len(df_test)

(12549, 9411, 3138, 3138)

In [30]:
# Use DictVectorizer(sparse=True) to turn the dataframe into matrices.
from sklearn.feature_extraction import DictVectorizer

In [32]:
def prepare_dfs(df):
    df = df.copy()
    
    df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
    df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)

    y_train_full = df_train_full['median_house_value'].values
    y_train = df_train['median_house_value'].values
    y_val = df_val['median_house_value'].values
    y_test = df_test['median_house_value'].values

    del df_train_full['median_house_value']
    del df_train['median_house_value']
    del df_val['median_house_value']
    del df_test['median_house_value']

    return df_train_full, df_train, df_val, df_test, y_train_full, y_train, y_val, y_test

    



## Question 1

Let's train a decision tree regressor to predict the `median_house_value` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?

* `ocean_proximity`
* `total_rooms`
* `latitude`
* `population`


In [33]:
df_train_full, df_train, df_val, df_test, y_train_full, y_train, y_val, y_test = prepare_dfs(df)

In [38]:
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text

In [44]:
dv = DictVectorizer(sparse=False)

dicts_train = df_train.to_dict(orient='records')
dicts_val = df_val.to_dict(orient='records')

X_train = dv.fit_transform(dicts_train)
X_val = dv.transform(dicts_val)

dt = DecisionTreeRegressor(max_depth=1) 
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

0.45168599736547216

In [45]:
print(export_text(dt, feature_names=dv.feature_names_))

|--- ocean_proximity=<1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=<1H OCEAN >  0.50
|   |--- value: [12.30]



answer: ocean_proximity=<1H OCEAN

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on validation?

* 0.045
* 0.245
* 0.545
* 0.845

In [46]:
from sklearn.ensemble import RandomForestRegressor

In [48]:
# dv = DictVectorizer(sparse=False)

# dicts_train = df_train.to_dict(orient='records')
# dicts_val = df_val.to_dict(orient='records')

# X_train = dv.fit_transform(dicts_train)
# X_val = dv.transform(dicts_val)

rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1) 
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

0.24472888684076877

answer: 0.245

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?

- 10
- 25
- 50
- 160


In [78]:
for n in range(10, 201, 10):
    # dv = DictVectorizer(sparse=False)

    # dicts_train = df_train.to_dict(orient='records')
    # dicts_val = df_val.to_dict(orient='records')
    
    # X_train = dv.fit_transform(dicts_train)
    # X_val = dv.transform(dicts_val)
    
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1) 
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = round(mean_squared_error(y_val, y_pred, squared=False), 3)
    
    print(f"n_estimators: {n} ---->  {rmse}")
    print()

n_estimators: 10 ---->  0.245

n_estimators: 20 ---->  0.238

n_estimators: 30 ---->  0.236

n_estimators: 40 ---->  0.235

n_estimators: 50 ---->  0.234

n_estimators: 60 ---->  0.234

n_estimators: 70 ---->  0.234

n_estimators: 80 ---->  0.234

n_estimators: 90 ---->  0.234

n_estimators: 100 ---->  0.234

n_estimators: 110 ---->  0.234

n_estimators: 120 ---->  0.234

n_estimators: 130 ---->  0.234

n_estimators: 140 ---->  0.234

n_estimators: 150 ---->  0.234

n_estimators: 160 ---->  0.233

n_estimators: 170 ---->  0.233

n_estimators: 180 ---->  0.234

n_estimators: 190 ---->  0.234

n_estimators: 200 ---->  0.234



answer: 160

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`


What's the best `max_depth`:

* 10
* 15
* 20
* 25


In [54]:
min_rmse = float('inf')

for d in [10, 15, 20, 25]:
    print(f"max_depth: {d}")
    print()
    rmses = []
    
    for n in range(10, 201, 10):
    
    
        rf = RandomForestRegressor(n_estimators=n, max_depth=d, random_state=1, n_jobs=-1) 
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = round(mean_squared_error(y_val, y_pred, squared=False), 5)
        if rmse < min_rmse:
            min_rmse = rmse
            res = (d, n, rmse)
        rmses.append(rmse)
        
        print(f"    n_estimators: {n:<10} ---->  {rmse:.5f}")
        # print()
    avg_rmse = np.mean(rmses)
    print(f"avg rmse {avg_rmse}")

best_md, best_n_esitmators, best_rmse = res
print()
print(f"The lowest rmse was {best_rmse} at a max_depth of {best_md}, and n_esitmators of {best_n_esitmators}")
    

max_depth: 10

    n_estimators: 10         ---->  0.25051
    n_estimators: 20         ---->  0.24726
    n_estimators: 30         ---->  0.24626
    n_estimators: 40         ---->  0.24509
    n_estimators: 50         ---->  0.24562
    n_estimators: 60         ---->  0.24547
    n_estimators: 70         ---->  0.24544
    n_estimators: 80         ---->  0.24561
    n_estimators: 90         ---->  0.24550
    n_estimators: 100        ---->  0.24538
    n_estimators: 110        ---->  0.24527
    n_estimators: 120        ---->  0.24500
    n_estimators: 130        ---->  0.24476
    n_estimators: 140        ---->  0.24459
    n_estimators: 150        ---->  0.24455
    n_estimators: 160        ---->  0.24445
    n_estimators: 170        ---->  0.24439
    n_estimators: 180        ---->  0.24449
    n_estimators: 190        ---->  0.24469
    n_estimators: 200        ---->  0.24473
avg rmse 0.245453
max_depth: 15

    n_estimators: 10         ---->  0.24553
    n_estimators: 20        

answer: based on the average the best max_depth would be 25

# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `total_rooms`
* `median_income`
* `total_bedrooms`
* `longitude`

In [55]:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1) 
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
rmse = round(mean_squared_error(y_val, y_pred, squared=False), 5)
rmse

0.24456

In [59]:
feature_importances = rf.feature_importances_

for feature, importance in zip(dv.feature_names_, feature_importances):
    print(f"{feature:<30} ---->  {importance:.5f}")

households                     ---->  0.01490
housing_median_age             ---->  0.03004
latitude                       ---->  0.10272
longitude                      ---->  0.08579
median_income                  ---->  0.33551
ocean_proximity=<1H OCEAN      ---->  0.21882
ocean_proximity=INLAND         ---->  0.14746
population                     ---->  0.02821
total_bedrooms                 ---->  0.01523
total_rooms                    ---->  0.02132


answer: median_income

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value


In [61]:
import xgboost as xgb

In [62]:
print(xgb.__version__)

1.7.6


In [64]:
dv.feature_names_

['households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'ocean_proximity=<1H OCEAN',
 'ocean_proximity=INLAND',
 'population',
 'total_bedrooms',
 'total_rooms']

In [67]:
dv.get_feature_names_out()

array(['households', 'housing_median_age', 'latitude', 'longitude',
       'median_income', 'ocean_proximity=<1H OCEAN',
       'ocean_proximity=INLAND', 'population', 'total_bedrooms',
       'total_rooms'], dtype=object)

In [72]:
features = dv.get_feature_names_out()
features = []
for f in dv.feature_names_:
	string = f.replace('=<', '-le')
	features.append(string)

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [75]:
features

['households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'ocean_proximity-le1H OCEAN',
 'ocean_proximity=INLAND',
 'population',
 'total_bedrooms',
 'total_rooms']

In [76]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=1,
                  evals=watchlist)

[0]	train-rmse:8.07362	val-rmse:8.07348
[1]	train-rmse:5.65832	val-rmse:5.65617
[2]	train-rmse:3.96917	val-rmse:3.96541
[3]	train-rmse:2.78836	val-rmse:2.78530
[4]	train-rmse:1.96409	val-rmse:1.96088
[5]	train-rmse:1.38983	val-rmse:1.38852
[6]	train-rmse:0.99273	val-rmse:0.99253
[7]	train-rmse:0.71854	val-rmse:0.72138
[8]	train-rmse:0.53440	val-rmse:0.54054
[9]	train-rmse:0.41016	val-rmse:0.42108
[10]	train-rmse:0.33195	val-rmse:0.34802
[11]	train-rmse:0.28413	val-rmse:0.30539
[12]	train-rmse:0.25487	val-rmse:0.28044
[13]	train-rmse:0.23748	val-rmse:0.26681
[14]	train-rmse:0.22789	val-rmse:0.25955
[15]	train-rmse:0.22197	val-rmse:0.25614
[16]	train-rmse:0.21543	val-rmse:0.25268
[17]	train-rmse:0.21034	val-rmse:0.24871
[18]	train-rmse:0.20688	val-rmse:0.24786
[19]	train-rmse:0.20341	val-rmse:0.24612
[20]	train-rmse:0.20036	val-rmse:0.24508
[21]	train-rmse:0.19827	val-rmse:0.24376
[22]	train-rmse:0.19644	val-rmse:0.24257
[23]	train-rmse:0.19320	val-rmse:0.24095
[24]	train-rmse:0.19203	va

In [77]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

watchlist = [(dtrain, 'train'), (dval, 'val')]

model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  verbose_eval=1,
                  evals=watchlist)

[0]	train-rmse:10.37456	val-rmse:10.37545
[1]	train-rmse:9.33895	val-rmse:9.33910
[2]	train-rmse:8.40699	val-rmse:8.40694
[3]	train-rmse:7.56827	val-rmse:7.56760
[4]	train-rmse:6.81351	val-rmse:6.81222
[5]	train-rmse:6.13433	val-rmse:6.13236
[6]	train-rmse:5.52318	val-rmse:5.52066
[7]	train-rmse:4.97329	val-rmse:4.97019
[8]	train-rmse:4.47854	val-rmse:4.47532
[9]	train-rmse:4.03346	val-rmse:4.03005
[10]	train-rmse:3.63299	val-rmse:3.62939
[11]	train-rmse:3.27269	val-rmse:3.26930
[12]	train-rmse:2.94855	val-rmse:2.94514
[13]	train-rmse:2.65703	val-rmse:2.65359
[14]	train-rmse:2.39479	val-rmse:2.39150
[15]	train-rmse:2.15901	val-rmse:2.15616
[16]	train-rmse:1.94714	val-rmse:1.94442
[17]	train-rmse:1.75677	val-rmse:1.75478
[18]	train-rmse:1.58567	val-rmse:1.58404
[19]	train-rmse:1.43206	val-rmse:1.43081
[20]	train-rmse:1.29412	val-rmse:1.29329
[21]	train-rmse:1.17023	val-rmse:1.17013
[22]	train-rmse:1.05934	val-rmse:1.05983
[23]	train-rmse:0.95998	val-rmse:0.96117
[24]	train-rmse:0.87015	

answer: 0.3 gives the best val-rmse